## 0. _Pre-processing_ 

In [1]:
import pandas as pd
import numpy as np

In [2]:
%%time

bottle_full = pd.read_csv('bottle.csv')

<string>:2: DtypeWarning: Columns (47,73) have mixed types. Specify dtype option on import or set low_memory=False.


Wall time: 57.1 s


In [3]:
bottle_full.shape

(864863, 74)

In [4]:
bottle_full.get_dtype_counts()

float64    65
int64       5
object      4
dtype: int64

In [5]:
bottle = bottle_full.dropna(0, subset=['T_degC'])
bottle = bottle.select_dtypes(include=[np.float64])
bottle.shape

(853900, 65)

## 1. _Quality metrics_ 

Я решил повторить действия из [семинара 5](https://github.com/HolyBayes/HSE_ML_2018-2019/blob/master/Seminars/Lesson_5.ipynb) и использовать модель линейной регрессии `ElasticNet`.
Но здесь проблема в том, что у `ElasticNetCV` нет параметра `scoring` и туда сейчас просто [зашит](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html#sklearn.linear_model.ElasticNetCV.score) _коэффициент детерминации_ $R^2$, то есть, в данном случае возможности выбора метрики для кросс-валидации нет.

Из постановки задачи $-$ предсказание температуры воды $-$ не получается сделать вывод о необходимости асимметричной метрики (ошибка предсказания температуры в любую сторону выглядит равнозначной).

Для интереса, при оценке качества предсказания на тестовом датасете попробую также применить метрику MAE.

In [6]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

## 2. _Regression models_ 

In [7]:
def model_fit_predict(model_pipeline, X_train, y_train, X_test, y_test):
    model = model_pipeline.fit(X_train, y_train)
    print('l1_ratio: {:.2f}, n_iter: {}'.format(model.named_steps['model'].l1_ratio_,model.named_steps['model'].n_iter_))
    y_pred = model_pipeline.predict(X_test)
    print('R2: {:.3f}'.format(r2_score(y_test, y_pred)))
    print('MAE: {:.3f}'.format(mean_absolute_error(y_test, y_pred)))
    print('MSE: {:.3f}'.format(mean_squared_error(y_test, y_pred)))

### _2.1 Preparation_

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [9]:
X, y = bottle.drop('T_degC',  1), bottle['T_degC']

In [10]:
imputer_scaler = Pipeline([
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='mean')),
    ('scaling', StandardScaler())
])

X_scaled = imputer_scaler.fit_transform(X, y)

#### _2.1.1 PCA_

Если простую линейную регрессию ещё можно посчитать на полном датасете, то, для того чтобы добавить комбинации признаков более высоких порядков, предварительно надо сжать простанство признаков.

In [11]:
from sklearn.decomposition import PCA

In [12]:
pca = PCA(n_components=12)
X_pca = pca.fit_transform(X_scaled)

In [13]:
for n in range(len(pca.explained_variance_ratio_), 0, -3):
    print( 'Explained variance for {} components: {:.3f}'.format(n, sum(pca.explained_variance_ratio_[:n])) )

Explained variance for 12 components: 0.675
Explained variance for 9 components: 0.592
Explained variance for 6 components: 0.488
Explained variance for 3 components: 0.349


#### _2.1.2 Splitting_

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, X_scaled_train, X_scaled_test, X_pca_train, X_pca_test, \
    y_train, y_test = train_test_split(X, X_scaled, X_pca, y, test_size=0.2, random_state=223)

In [16]:
X_train.shape, X_test.shape

((683120, 64), (170780, 64))

### _2.2 Linear_

In [17]:
from sklearn.linear_model import ElasticNetCV

In [18]:
linear_model = Pipeline([
    ('model', ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=4, n_jobs=4, max_iter=512))
])

#### _2.2.1 Linear regression on PCA-reduced dataset_

In [19]:
%%time

model_fit_predict(linear_model, X_pca_train, y_train, X_pca_test, y_test)

l1_ratio: 1.00, n_iter: 3
R2: 0.783
MAE: 1.353
MSE: 3.916
Wall time: 59.9 s


#### _2.2.2 Linear regression on full dataset_

In [20]:
%%time

model_fit_predict(linear_model, X_scaled_train, y_train, X_scaled_test, y_test)

l1_ratio: 1.00, n_iter: 369
R2: 1.000
MAE: 0.004
MSE: 0.000
Wall time: 4min 26s


### _2.3 Poly$^2$_

In [21]:
from sklearn.preprocessing import PolynomialFeatures

In [22]:
%%time

poly2_model = Pipeline([
    ('features', PolynomialFeatures(2)),
    ('model', ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=4, n_jobs=4, max_iter=8192))
])

model_fit_predict(poly2_model, X_pca_train, y_train, X_pca_test, y_test)

l1_ratio: 0.70, n_iter: 232
R2: 0.790
MAE: 1.341
MSE: 3.788
Wall time: 6min 20s


### _2.4 Poly$^3$_

Пришлось ограничить размер тренингового датасета, потому что иначе очень долго считает.

In [23]:
_, X_short_pca_train, _, y_short_train = train_test_split(X_pca_train, y_train, test_size=0.2, random_state=229)

In [24]:
%%time

poly3_model = Pipeline([
    ('features', PolynomialFeatures(3)),
    ('model', ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=4, n_jobs=4, max_iter=131072))
])

model_fit_predict(poly3_model, X_short_pca_train, y_short_train, X_pca_test, y_test)

l1_ratio: 1.00, n_iter: 2
R2: 0.000
MAE: 3.489
MSE: 18.031
Wall time: 4min 46s


В итоге ничего хорошего и не насчитал)
Модель с кубическими комбинациями признаков явно не подходит для нашего датасета.

### _2.5 FunctionTransformer_

In [25]:
from sklearn.preprocessing import FunctionTransformer

In [26]:
%%time

ft_model = Pipeline([
    ('transformer', FunctionTransformer(np.tanh, validate=True)),
    ('model', ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=4, n_jobs=4, max_iter=4096))
])

model_fit_predict(ft_model, X_pca_train, y_train, X_pca_test, y_test)

l1_ratio: 1.00, n_iter: 40
R2: 0.730
MAE: 1.657
MSE: 4.872
Wall time: 55.7 s


## 3. _Scaling required?_

Модели, имеющие в основе `ElasticNet` - линейную модель с регуляризацией - требуют стандартизованных данных.

`rbf` ядро `SVR` рассчитывает расстояния между многомерными векторами, и, соответственно, также должно использовать стандартизованные данные.

Вообще, насколько я понял, для простой линейной регрессии стандартизация технически не требуется, хотя, скорее всего, для более очевидной интерпретации результатов (весов признаков), её лучше делать в любом случае, потому что делением на СКО мы фактически делаем величину безразмерной.

## 4. _Overall quality_

Как можно видеть по метрикам качества, обычная линейная регрессия на полном наборе признаков даёт почти идеальное предсказание. Так что с этой точки зрения можно утверждать, что существует очевидная зависимость температуры воды от химических показателей (хотя, вероятно, тут можно порассуждать, насколько применение `Imputer(mean)` могло на это повлиять).

Что касается сжатого с помощью **PCA** датасета, то здесь, видимо, можно сделать два вывода:
1. Даже на сокращённом датасете качество предсказания целевой переменной достаточно приемлемое.
2. В данном случае, видимо, особого смысла добавлять признаки более высоких порядков нет, потому что вычислительная сложность увеличивается, а существенного прироста точности предсказания это не даёт. 